<a href="https://colab.research.google.com/github/igedeaguss/Magang-Pharos/blob/main/Recommendation_System_Ethical_Product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymssql
!pip install rapidfuzz
import pymssql
import pandas as pd
import numpy as np
from google.colab import auth
from google.auth import default
import re
from rapidfuzz import fuzz, process
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe
from google.auth import default

#Mount Google Drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.7 MB/s eta 0:00:00
Mounted at /content/drive


# All Data

In [2]:
#---Data untuk Pencocokan---
data_ims = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/IMS Compiled Q3 2019 - Q2 2024 (List Product).xlsx', sheet_name='Sheet1')
data_survey = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Hospital Survey Product Name Pivot.xlsx', sheet_name='Sheet1')
data_pi_2024_2025 = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Produk PI Sales 2024 - 2025.xlsx', sheet_name='Sheet1', dtype={'Item Kode':str})


#---Data untuk Mapping Bentuk Sediaan
# Dapatkan kredensial dan buat klien gspread
creds, _ = default()
gc = gspread.authorize(creds)

# Buka Google Sheet dengan URL atau ID
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1ztO5ypZI3K99wvvTIaUny4PF2VG4tibYSckQR25JPqY'

# Ambil spreadsheet
spreadsheet = gc.open_by_url(spreadsheet_url)

# Pilih worksheet
worksheet = spreadsheet.worksheet('Data Final Sediaan')  # atau gunakan nama worksheet seperti spreadsheet.worksheet('Sheet1')

# Dapatkan data sebagai DataFrame
data_sediaan = get_as_dataframe(worksheet)

data_survey_= pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/06012025 Raw Data Hospital Survey Final.xlsx', sheet_name='Final')

#---Data untuk Sistem Rekomendasi
url_dokter_survey = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NIOYmajDgkSY7lfKNH_wHRz0I80eeZX2OWhNtAdMw-8/edit?gid=0#gid=0')
url_database_ethical = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Qwurf3wlLDbrhWNB0XX1CgpK7GuHqCHZoAlQK_GJqek/edit?usp=sharing')
url_data_score_dokter = gc.open_by_url('https://docs.google.com/spreadsheets/d/1wIe9UOG8F6aNy1PF1UzGNQnkrcCm6DAD5zjUEnMbUo8/edit?usp=sharing')
url_data_survey_final_pm = gc.open_by_url('https://docs.google.com/spreadsheets/d/1i4mcLMshz82J1PABVtzA_tuuvdJU9Mfm7Y0qOL7grjI/edit?gid=667170748#gid=667170748')

#Koneksi ke Database
connection = pymssql.connect(server='35.187.243.174', user='tim_powerbi',
                             password='dGltcG93ZXJiaQ', database='mkt_insight', port = '1433')

#Menarik data kode PI dan item kode
query_1 = """
SELECT DISTINCT KodePI, [Item Kode]
FROM mkt_insight.dbo.vw_DIR10001B
WHERE KodePI IS NOT NULL
  AND KodePI <> ''
  AND (Periode LIKE '%2023%' OR Periode LIKE '%2024%')
"""


#Menarik data yang menghasilkan item kode, nama produk, zat aktif, dan total penjualan
query_2 = """
SELECT z.[Nama Produk], s.[Item Kode], z.[Zat Aktif], SUM(s.[Value Sales]) AS [Total Penjualan]
FROM mkt_insight.dbo.DIR10001B s
INNER JOIN mkt_insight.dbo.ListAllProductComplete z ON s.[Item Kode]= z.[Kode Produk PI]
GROUP BY s.[Item Kode], z.[Nama Produk], z.[Zat Aktif]"""


#Menarik data nama outlet dan kode PI
query_3 = """
SELECT DISTINCT(KodePI), NamaOutlet, Propinsi, Alamat
FROM mkt_insight.dbo.Struktur_Marketing_PI
"""

# Menarik Data PSSP
query_4 = """
SELECT * FROM mkt_insight.dbo.PSSP_PIdanPML
"""

'''
data_1 = pd.DataFrame(pd.read_sql_query(query_1, connection))
data_2 = pd.DataFrame(pd.read_sql_query(query_2, connection))'''

# Dataset untuk mencocokan nama outlet terdaftar
data_struktur_PI = pd.DataFrame(pd.read_sql_query(query_3, connection))

#Dataset produk dan zat aktif perusahaan
data_produk_zat_aktif = pd.DataFrame(pd.read_sql_query(query_2, connection))

#Dataset PSSP
data_pssp = pd.DataFrame(pd.read_sql_query(query_4, connection))

#Dataset standarisasi
data_standarisasi = pd.DataFrame(pd.read_sql_query(query_1, connection))

# Menutup koneksi
connection.close()

#Dataset berisi nama dokter dan kode dokter yang terdata di Database
data_dokter = get_as_dataframe(url_dokter_survey.worksheet('DB Dokter'), evaluate_formulas=True)
data_dokter = data_dokter.loc[:, ~data_dokter.columns.str.contains('^Unnamed')]

# Matching Name Product

## Data

In [3]:
data_prdk_survey = data_survey.copy()
data_ims_prdk = data_ims.copy()
data_pi_prdk = data_pi_2024_2025.copy()

## Cleaning Data

In [4]:
#Membersihkan nama brand IMS
def clean_product_name(name):
    return name.split(' -')[0].strip().lower() if isinstance(name, str) else ""

# Daftar bentuk sediaan yang diakui
BENTUK_SEDIAAN = [
    "TAB", "CAP", "SYRUP", "SYR", "INJ", "CREAM", "GEL", "SUSP", "EFF", "ODT",
    "PWD", "OINT", "SUPPOS", "INF", "DRP", "CONC", "SOL", "AMP", "ML", "MG", "IU", "MCG", "G"
]

def clean_product_name_survey(product_name):
    """
    Membersihkan nama produk dengan aturan:
    1. Menghapus produk jika mengandung kata 'racikan' atau 'racik'.
    2. Menghapus teks dalam tanda kurung, kecuali jika mengandung angka atau bentuk sediaan.
    3. Jika tanda kurung berisi angka atau bentuk sediaan, hapus tanda kurungnya tetapi biarkan isinya.
    """
    if not isinstance(product_name, str) or product_name.strip() == '':
        return None  # Abaikan jika input kosong atau bukan string

    product_name = product_name.strip()

    # Jika nama produk mengandung "racikan" atau "racik", maka diabaikan (return None)
    if re.search(r'\bracik(an)?\b', product_name, re.IGNORECASE):
        return None

    # Fungsi untuk memproses tanda kurung
    def process_parentheses(match):
        content = match.group(1).strip()
        contains_number = bool(re.search(r'\d', content))  # Cek apakah ada angka
        contains_form = any(sediaan in content.upper() for sediaan in BENTUK_SEDIAAN)  # Cek apakah ada bentuk sediaan

        if contains_number or contains_form:
            return f" {content} "  # Hilangkan tanda kurung, tetapi biarkan isinya
        return ''  # Hapus jika tidak mengandung angka atau bentuk sediaan

    # Hapus tanda kurung yang tidak mengandung angka/bentuk sediaan atau hilangkan kurungnya jika cocok
    cleaned_name = re.sub(r'\((.*?)\)', process_parentheses, product_name)

    # Hilangkan spasi ganda yang mungkin muncul
    cleaned_name = re.sub(r'\s+', ' ', cleaned_name).strip()

    return cleaned_name


#Standarisasi bentuk sediaan
def extract_dosage_form(product_name):
    dosage_form_map = {
        r'\btabs?\b|\btablets?\b|\btablet\b': 'TAB',
        r'\bcaps?\b|\bcapsules?\b|\bcapsul\b': 'CAP',
        r'\bkapsul\b|\bkapsule\b|\bkap\b': 'CAP',
        r'\bsuspensi\b|\bsuspension\b|\bsusp\b': 'SUSP',
        r'\bserbuk\b|\bbubuk\b|\bpowder\b|\bpwd\b': 'PWD',
        r'\boint\b|\bointment\b|\bsalep\b': 'OINT',
        r'\bkaplet\b|\bcaplets?\b|\bkapl\b|\bcaplet\b': 'CAP',
        r'\binj\b|\binjeksi\b|\binjection\b': 'INJ',
        r'\bamp\b|\bampule\b|\bampul\b': 'AMP',
        r'\bcream\b|\bcrm\b|\bkrim\b|\bcr\b': 'CRM',
        r'\bsupp\b|\bsuppositoria\b|\bsuppos\b': 'SUPPOS',
        r'\binf\b|\binfusion\b|\binfus\b': 'INF',
        r'\bsyrup\b|\bsirup\b|\bsyr\b': 'SYR',
        r'\bdrop\b|\bdrops\b|\btetes\b': 'DRP',
        r'\bconc\b|\bconcentrate\b': 'CONC',
        r'\bsol\b|\bsolution\b': 'SOL',
        r'\bgel\b': 'GEL'
    }

    if not isinstance(product_name, str) or product_name.strip() == '':
        return ''

    # Ubah bentuk sediaan dalam nama produk
    for pattern, replacement in dosage_form_map.items():
        product_name = re.sub(pattern, replacement, product_name, flags=re.IGNORECASE)

    return product_name  # Tidak langsung uppercase agar mudah dibaca dalam tahap sebelumnya

# Standarisasi: Ubah "syringe" menjadi "inj" dalam data IMS
data_ims_prdk["Standardized Name"] = data_ims_prdk["Pack"].str.replace(r"\bsyringe\b", "inj", regex=True)


# Kata kunci alat kesehatan
medical_device_keywords = [
    "stop cock", "catheter", "stent", "scalpel", "forceps", "retractor",
    "bandage", "suture", "gloves", "gown", "dental", "lens", "surgical", "probe", "monitor",
    "electrode", "defibrillator", "ventilator", "prosthesis", "wire", "needle", "screw", "plate",
    "straw", "catether", "selang", "stopcock", "incisor", "sponge", "locking", "accu check", "accucheck","accu-check", "accu-cek"
    "arm sling", "action reliever", "active lite", "adult bacterial", "oxygenator", "adult rem", "selang",'fillter','filter'
    "advantime", "aesculap", "affinity", "alas", "alcohol swab", "allevyn adhesive","face mask", "gynecology set","alert snap", 'baju','mapping',
    "selectable reload","promus premier","abbocath",'guedel air way','handskoen maxter','jarum','kasa','sabun','masker','pembalut siku','sarung tangan',
    "spuit",'topi','urine bag','underpad','under pad','syringe with needle','syringe without needle'
]

# Fungsi untuk mengecek apakah produk alat kesehatan
def is_medical_device(product_name):
    """Cek apakah nama produk merupakan alat kesehatan"""
    product_name_lower = str(product_name).lower()

    # Jika produk ada di IMS, maka bukan alat kesehatan
    if product_name in data_ims_prdk["Pack"].values:
        return False

    # Jika mengandung kata kunci alat kesehatan, maka termasuk alat kesehatan
    return any(keyword in product_name_lower for keyword in medical_device_keywords)

#Standarisasi Nama Produk di Dataset Utama
data_ims_prdk["Standardized Name"] = data_ims_prdk["Pack"].apply(lambda x:
    data_ims_prdk.loc[data_ims_prdk["Pack"] == x, "Standardized Name"].values[0]
    if x in data_ims_prdk["Pack"].values else x
)

def remove_if_contains(text):
    if pd.isna(text):  # Jika nilainya NaN, langsung return
        return text

    # Pola regex untuk mendeteksi kata "compound" atau "racikan"
    pattern = r'\b(compound|racikan|racik)\b'

    # Jika ditemukan kata tersebut, ubah menjadi NaN
    if re.search(pattern, text, re.IGNORECASE):
        return pd.NA  # Gunakan pd.NA untuk mengisi NaN
    else:
        return text  # Jika tidak cocok, kembalikan teks aslinya


# Terapkan ke dataset
data_ims_prdk['Cleaned Brand Name'] = data_ims_prdk['Product'].apply(clean_product_name)

data_prdk_survey['Cleaned Name'] = data_prdk_survey['Product Name'].apply(clean_product_name_survey)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Cleaned Name'].apply(lambda x: x if not is_medical_device(x) else pd.NA)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Cleaned Name'].apply(extract_dosage_form)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Cleaned Name'].apply(remove_if_contains)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Cleaned Name'].str.upper()


data_pi_prdk['Cleaned Name'] = data_pi_prdk['Nama Produk'].apply(extract_dosage_form)

## Matching Name Product Algorithm

### Data IMS and PI Product

In [5]:
# Fungsi untuk mengekstrak angka dan unit dari nama produk
def extract_numbers_and_units(name):
    matches = re.findall(r'(\d+\.?\d*)\s?([A-Za-z]*)', name)  # Tangkap angka + unit
    return {f"{num} {unit}".strip() for num, unit in matches}  # Gunakan set agar unik

# Fungsi untuk mencocokkan angka dengan unit
def match_numbers_with_units(survey_numbers, pack_numbers):
    if not survey_numbers or not pack_numbers:
        return 0  # Jika salah satu kosong, skor = 0

    survey_set = set(survey_numbers)
    pack_set = set(pack_numbers)
    intersection = survey_set.intersection(pack_set)

    # Skor dihitung berdasarkan banyaknya pasangan angka + satuan yang cocok
    score = (len(intersection) / max(len(survey_set), len(pack_set))) * 100
    return score

# Fungsi untuk mencocokkan bentuk sediaan
def match_dosage_form(survey_str, ims_str):
    common_forms = ["SYRUP", "SYR DRY", "TAB", "CAP", "INJ", "POWDER", "PWD", "SYR", "CAP",
                    "CRM", "SUSP", "AMP", "OINT", "DRP", "CONC", "SOL", "GEL"]

    # Temukan bentuk sediaan dalam kedua string
    survey_form = next((form for form in common_forms if form in survey_str.upper()), "")
    ims_form = next((form for form in common_forms if form in ims_str.upper()), "")

    # Jika bentuk sediaan ada di kedua produk, gunakan fuzzy matching
    if survey_form and ims_form:
        return fuzz.partial_ratio(survey_form, ims_form)

    return 50  # Jika tidak ada bentuk sediaan, beri skor default rendah


def calculate_similarity(cleaned_survey_name, pack_list):
    results = []
    survey_numbers = extract_numbers_and_units(cleaned_survey_name)  # Pastikan string

    for pack_tuple in pack_list:
        pack_name, molecule, atc4 = pack_tuple  # Unpack tuple lebih awal
        pack_numbers = extract_numbers_and_units(pack_name)  # Sekarang ini string

        # Hitung skor kemiripan angka + satuan
        score_numbers = match_numbers_with_units(survey_numbers, pack_numbers)

        # Hitung skor kemiripan string (nama produk)
        score_strings = fuzz.token_sort_ratio(cleaned_survey_name, pack_name)

        # Hitung kemiripan bentuk sediaan
        score_dosage_form = match_dosage_form(cleaned_survey_name, pack_name)

        # Gabungkan skor dengan bobot (angka lebih penting)
        combined_score = (0.5 * score_numbers) + (0.4 * score_strings) + (0.1 * score_dosage_form)
        results.append((pack_name, molecule, atc4, combined_score))

    return results

# Fungsi utama untuk memproses data dengan filter Cleaned Brand
def process_brand_matching(data_ims, data_survey):
    results = []

    # Ambil daftar nilai unik dari 'Cleaned Brand Name' di IMS
    unique_brands = data_ims['Cleaned Brand Name'].dropna().unique()

    for brand_keyword in unique_brands:
        # Gunakan regex untuk filter survey yang mengandung brand_keyword
        filtered_survey = data_survey[
            data_survey['Nama Produk'].str.contains(fr"\b{re.escape(brand_keyword)}\b", case=False, na=False)
        ]

        # Ambil hanya produk IMS dengan brand yang sama
        filtered_ims = data_ims[data_ims['Cleaned Brand Name'] == brand_keyword]
        pack_list = list(zip(filtered_ims['Pack'], filtered_ims['Pack Molecule String'], filtered_ims['ATC4']))

        for _, survey_row in filtered_survey.iterrows():
            cleaned_survey_name = survey_row['Cleaned Name']
            produk_survey = survey_row['Nama Produk']
            kode_pi = survey_row['Item Kode']

            # Menghitung skor kemiripan
            similarity_scores = calculate_similarity(cleaned_survey_name, pack_list)

            # Cari produk IMS dengan skor tertinggi
            if similarity_scores:
                best_match_pack, best_molecule, best_atc4, best_score = max(similarity_scores, key=lambda x: x[3])

                # Simpan hasil terbaik
                results.append({
                    'Kode PI': kode_pi,
                    'Product PI': produk_survey,
                    'Best Match IMS Pack': best_match_pack,
                    'Pack Molecule String': best_molecule,
                    'ATC4': best_atc4,
                    'Best Similarity Score': best_score
                })

    # Buat DataFrame hasil
    results_df = pd.DataFrame(results)
    return results_df.sort_values(by='Best Similarity Score', ascending=False)

# Contoh penggunaan
filtered_results_pi = process_brand_matching(data_ims_prdk, data_pi_prdk)

hasil = filtered_results_pi.copy()
data_final_pi = data_pi_2024_2025.merge(hasil,
                                        left_on='Item Kode',
                                        right_on='Kode PI',
                                        how='left')
data_final_pi = data_final_pi[['Item Kode','Nama Produk','Best Match IMS Pack','Best Similarity Score','Pack Molecule String','ATC4','Total Penjualan','Group Brand']]
data_final_pi

,Item Kode,Nama Produk,Best Match IMS Pack,Best Similarity Score,Pack Molecule String,ATC4,Total Penjualan,Group Brand
0,009100,HERCUM 0.05 % CREAM 10 G,HERCUM CRM 0.05% 10G,94.285714,CLOBETASOL,D07A0 - TOP CORTICOSTEROIDS PLAIN,9.276000e+08,HERCUM
1,013852,ACUCEL 0.1% GEL 10 G,NaN,NaN,NaN,NaN,-8.800000e+04,ACUCEL
2,008220,IFLACORT CREAM 5 G,IFLACORT CRM 0.1% 5G,66.111111,MOMETASONE,D07A0 - TOP CORTICOSTEROIDS PLAIN,4.093264e+08,IFLACORT
3,027173,AYU DERMA WHITENING CARE DAY CREAM 30 G,NaN,NaN,NaN,NaN,2.411600e+07,AYU DERMA
4,029670,NOURISH ACNE PLAST GIRL PROMO BUNDLING,NaN,NaN,NaN,NaN,1.018624e+09,ACNE PLAST
...,...,...,...,...,...,...,...,...
369,027536,SILKY WHITE WHITENING DAY CREAM 25 G,NaN,NaN,NaN,NaN,2.052610e+07,SILKY WHITE
370,012917,QUINOBIOTIC 200 MG/100 ML INFUSION,QUINOBIOTIC TAB 250MG 30,30.660377,CIPROFLOXACIN,J01G1 - ORAL FLUOROQUINOLONES,7.063620e+08,QUINOBIOTIC
371,004810,ANTIPRESTIN 20 KAPSUL,ANTIPRESTIN CAP 20MG 30,45.121951,FLUOXETINE,N06A4 - SSRI ANTIDEPRESSANTS,1.201197e+09,ANTIPRESTIN
372,006108,TRICHOSTATIC OVULA,TRICHOSTATIC VAG SUPP 6,32.317073,METRONIDAZOLE+NYSTATIN,G01A2 - TRICHOMONACIDE TOPICAL,1.179210e+08,TRICHOSTATIC


#### Save to GDrive

In [6]:
import os

file_name = f"Data Hasil Match Product (PI and IMS).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Match Product"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
data_final_pi.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Match Product/Data Hasil Match Product (PI and IMS).xlsx


### Data Survey and IMS

In [7]:
# 1️⃣ Fungsi untuk standarisasi nama produk
def standardize_product_name(name):
    if pd.isna(name):
        return ""

    # Hapus tanda kurung kecuali yang mengandung dosis atau satuan
    pattern = r"\((?!\d+\s?(ML|MG|IU|G|MCG|L|KG|M[LI]U|CAP|TAB|VIAL|AMP|PFS|SUP|DRP|TUBE|PATCH|SACH|BAG|BOTTLE))[^)]*\)"
    name = re.sub(pattern, "", name)

    # Hapus karakter khusus kecuali angka, huruf, '%', dan '/'
    name = re.sub(r"[^a-zA-Z0-9\s%/]", "", name)

    return name.strip()

# 2️⃣ Fungsi ekstraksi angka + satuan
def extract_numbers_and_units(name):
    matches = re.findall(r'(\d+\.?\d*)\s?([A-Za-z%/]*)', name)
    return {f"{num} {unit}".strip() for num, unit in matches}

# 3️⃣ Fungsi pencocokan angka + satuan
def match_numbers_with_units(survey_numbers, pack_numbers):
    if not survey_numbers or not pack_numbers:
        return 0

    intersection = survey_numbers.intersection(pack_numbers)
    return (len(intersection) / max(len(survey_numbers), len(pack_numbers))) * 100

# 4️⃣ Fungsi pencocokan bentuk sediaan
def match_dosage_form(survey_str, ims_str):
    common_forms = ["SYRUP", "SYR DRY", "TAB", "CAP", "INJ", "POWDER", "PWD", "SYR", "CRM", "SUSP",
                    "AMP", "OINT", "DRP", "CONC", "SOL", "GEL"]

    survey_form = next((form for form in common_forms if form in survey_str.upper()), "")
    ims_form = next((form for form in common_forms if form in ims_str.upper()), "")

    return fuzz.partial_ratio(survey_form, ims_form) if survey_form and ims_form else 50


# 6️⃣ Fungsi utama untuk menghitung kemiripan
def calculate_similarity(cleaned_survey_name, pack_list, brand_words):
    results = []
    survey_numbers = extract_numbers_and_units(cleaned_survey_name)

    for pack_tuple in pack_list:
        pack_name, molecule, brand, atc4 = pack_tuple
        pack_numbers = extract_numbers_and_units(pack_name)

        # Skor angka + satuan
        score_numbers = match_numbers_with_units(survey_numbers, pack_numbers)
        # Skor string berbasis fuzzy matching
        score_strings = fuzz.partial_ratio(cleaned_survey_name, pack_name)
        # Skor bentuk sediaan
        score_dosage_form = match_dosage_form(cleaned_survey_name, pack_name)

        # Gabungkan skor dengan bobot baru
        combined_score = (0.3 * score_numbers) + (0.5 * score_strings) + (0.2 * score_dosage_form)
        results.append((pack_name, molecule, brand, atc4, combined_score, len(pack_name)))

    # Urutkan berdasarkan skor kemiripan dan panjang nama
    results.sort(key=lambda x: (x[4], x[5]), reverse=True)
    return results

def process_brand_matching(data_ims, data_survey, similarity_threshold=90):
    results = []
    data_survey = data_survey[data_survey['Cleaned Name'].notna() & (data_survey['Cleaned Name'].str.strip() != "")]

    # Ambil data survey sebagai list of dictionaries
    data_survey_dict = data_survey.to_dict('records')

    unique_brands = data_ims['Cleaned Brand Name'].dropna().unique()

    for brand_keyword in unique_brands:
        if re.search(r'\bracikan\b|\bracik\b', brand_keyword, re.IGNORECASE):
            continue

        # 💡 **Gunakan kata yang bukan angka & lebih dari 3 karakter**
        brand_words = [word for word in brand_keyword.lower().split() if len(word) > 3 and not word.isnumeric()]

        if not brand_words:
            continue

        # Ambil produk IMS yang sesuai dengan brand ini
        filtered_ims = data_ims[data_ims['Cleaned Brand Name'] == brand_keyword]
        pack_list = list(zip(
            filtered_ims['Pack'],
            filtered_ims['Pack Molecule String'],
            filtered_ims['Cleaned Brand Name'],
            filtered_ims['ATC4']
        ))

        # 🔍 **Filter survey produk yang relevan dengan brand_words menggunakan RapidFuzz**
        filtered_survey = [
            row for row in data_survey_dict
            if any(fuzz.partial_ratio(brand_word, row['Cleaned Name'].lower()) >= similarity_threshold for brand_word in brand_words)
        ]

        if not filtered_survey:
            continue

        for survey_row in filtered_survey:
            cleaned_survey_name = survey_row['Cleaned Name']
            produk_survey = survey_row['Product Name']

            # Hitung kemiripan dengan produk IMS
            similarity_scores = calculate_similarity(cleaned_survey_name, pack_list, brand_words)

            # Ambil hasil terbaik
            if similarity_scores:
                best_match = similarity_scores[0]
                best_match_pack, best_molecule, best_brand, best_atc4, best_score, _ = best_match
                results.append({
                    'Product Survey': produk_survey,
                    'Best Match IMS Pack': best_match_pack,
                    'Pack Molecule String': best_molecule,
                    'Brand': best_brand,
                    'ATC4': best_atc4,
                    'Best Similarity Score': best_score
                })
            else:
                print(f"Tidak ada pencocokan untuk produk: {produk_survey}")

    return pd.DataFrame(results)


# 8️⃣ Eksekusi Program
filtered_results = process_brand_matching(data_ims_prdk, data_prdk_survey)

# Ambil hanya hasil dengan skor tertinggi untuk setiap produk survey
if not filtered_results.empty:
    filtered_results = filtered_results.loc[filtered_results.groupby('Product Survey')['Best Similarity Score'].idxmax()]


#Join data hasil match dengan data produk survey (unik)
data_final_survey = data_prdk_survey[['Product Name']].merge(filtered_results[['Product Survey','Best Match IMS Pack','Pack Molecule String','ATC4','Best Similarity Score']],
                                                                    left_on='Product Name',
                                                                    right_on='Product Survey',
                                                                    how='left')
data_final_survey = data_final_survey[['Product Name','Best Match IMS Pack','Pack Molecule String','ATC4','Best Similarity Score']]
data_final_survey

,Product Name,Best Match IMS Pack,Pack Molecule String,ATC4,Best Similarity Score
0,ATORVASTATIN 20MG TAB/AMAROX,ATORVASTATIN TAB FLM CTD 20MG 100 STRP,ATORVASTATIN,C10A1 - STATINS (HMG-COA RED),71.363636
1,BISOPROLOL 5 MG /HJ,BISOPROLOL TAB 5MG 30 STRP,BISOPROLOL,C07A0 - BETA BLOCKING AGENT PLN,64.473684
2,BISOPROLOL 5 MG TABLET,BISOPROLOL TAB FLM CTD 5MG 100 STRP,BISOPROLOL,C07A0 - BETA BLOCKING AGENT PLN,77.424242
3,CALOS KAPLET,CALOS TAB CHEW 500MG 30,CALCIUM,A12A0 - CALCIUM,49.176471
4,CAPTOPRIL 12.5 MG,CAPTOPRIL TAB 12.5MG 100 STRP,CAPTOPRIL,C09A0 - ACE INHIBITORS PLAIN,61.235294
...,...,...,...,...,...
50330,ZYVOX 600MG INJ 300ML,ZYVOX TAB 600MG 10,LINEZOLID,J01X9 - ALL OTHER ANTIBACTERIALS,55.000000
50331,ZYVOX 600MG TAB,ZYVOX TAB 600MG 10,LINEZOLID,J01X9 - ALL OTHER ANTIBACTERIALS,72.500000
50332,ZYVOX 600MG/300ML POLYBAG,ZYVOX POLYBAG 2MG/ML 300ML,LINEZOLID,J01X9 - ALL OTHER ANTIBACTERIALS,59.210526
50333,Zyvox Infusion 2 mg/mL - 300 mL,ZYVOX POLYBAG 2MG/ML 300ML,LINEZOLID,J01X9 - ALL OTHER ANTIBACTERIALS,76.538462


#### Save to GDrive

In [11]:
import os

file_name = f"Data Hasil Match Product (IMS & Survey).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Match Product"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
data_final_survey.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Match Product/Data Hasil Match Product (IMS & Survey).xlsx


# Mapping Dosage Form

## Load Data

In [11]:
data_pi = data_final_pi
data_survey = data_final_survey

## Mapping Dosage Form Alogrithm

In [12]:
def extract_dosage_form(product_name):
    dosage_form_map = {
        r'\btabs?\b|\btablets?\b|\btablet\b': 'tab',
        r'\bcaps?\b|\bcapsules?\b|\bcapsul\b': 'cap',
        r'\bkapsul\b|\bkapsule\b|\bkap\b': 'cap',
        r'\bsuspensi\b|\bsuspension\b|\bsusp\b': 'susp',
        r'\bserbuk\b|\bbubuk\b|\bpowder\b|\bpwd\b': 'pwd',
        r'\boint\b|\bointment\b|\bsalep\b': 'oint',
        r'\bkaplet\b|\bcaplets?\b|\bkapl\b|\bcaplet\b': 'cap',
        r'\binj\b|\binjeksi\b|\binjection\b': 'inj',
        r'\bamp\b|\bampule\b|\bampul\b': 'amp',
        r'\bcream\b|\bcrm\b|\bkrim\b|\bcr\b': 'crm',
        r'\bsupp\b|\bsuppositoria\b|\bsuppos\b': 'suppos',
        r'\binf\b|\binfusion\b|\binfus\b': 'inf',
        r'\bsyrup\b|\bsirup\b|\bsyr\b': 'syr',
        r'\bdrop\b|\bdrops\b|\btetes\b': 'drp',
        r'\bconc\b|\bconcentrate\b': 'conc',
        r'\bsol\b|\bsolution\b': 'sol',
        r'\bgel\b': 'gel'
    }
    for pattern, standard_form in dosage_form_map.items():
        if re.search(pattern, product_name, flags=re.IGNORECASE):
            return standard_form
    return ''

# Mapping bentuk sediaaan data PI
data_pi['Dosage Form'] = data_pi['Nama Produk'].apply(extract_dosage_form)

# Mapping bentuk sediaaan data Survey
data_survey['Dosage Form'] = data_survey['Product Name'].apply(extract_dosage_form)


merged_df_pi = pd.merge(data_pi, data_sediaan, left_on='Dosage Form', right_on='Form', how='left')
merged_df_pi.drop(columns=['Form'], inplace=True)

merged_df_survey = pd.merge(data_survey, data_sediaan, left_on='Dosage Form', right_on='Form', how='left')
merged_df_survey.drop(columns=['Form'], inplace=True)

merged_df_survey = merged_df_survey.drop_duplicates('Product Name')
data_survey_merged = pd.merge(data_survey_, merged_df_survey[['Product Name','Pack Molecule String','ATC4','Dosage Form']],
                              left_on='Product Name without #',
                              right_on='Product Name',
                              how='left')

## Save to GDrive

In [19]:
import os

file_name = f"Data Produk Survey (Final).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset untuk sistem rekomendasi"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
data_survey_merged.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Produk Survey (Final).xlsx


In [20]:
file_name = f"Data Produk PI (Final).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset untuk sistem rekomendasi"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
merged_df_pi.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Produk PI (Final).xlsx


# Recommendation System

## Load Dataset

In [13]:
#Dataset berisi nama dokter dan kode dokter yang terdata di Database
data_dokter = get_as_dataframe(url_dokter_survey.worksheet('DB Dokter'), evaluate_formulas=True)
data_dokter = data_dokter.loc[:, ~data_dokter.columns.str.contains('^Unnamed')]

#Dataset produk pharos beserta zat aktifnya
data_produk_pharos = merged_df_pi.copy()

data_survey_final = data_survey_merged.copy()
#data_survey_final =  pd.read_excel('/content/drive/MyDrive/Dataset Hasil Match Product/24022025 Data Hasil Match Product (IMS & Survey).xlsx',sheet_name='Versi 05032025')

data_survey_final_pm = get_as_dataframe(url_data_survey_final_pm.worksheet('Sheet1'), evaluate_formulas=True) #dari tim Ethical

## Cleaning Data and Prepare Data

In [14]:
data_survey_zat = data_survey_final.copy()

#Cleaning data survey
data_survey_zat.iloc[:, 4:16] =data_survey_zat.iloc[:, 4:16].fillna(0)
data_survey_zat = data_survey_zat[data_survey_zat['Pack Molecule String'].isnull() == False]
data_survey_zat['Pack Molecule String'] = data_survey_zat['Pack Molecule String'].str.replace('\n', '', regex=False).str.replace(', ', ',', regex=False)
data_survey_zat['Pack Molecule String'] = data_survey_zat['Pack Molecule String'].str.replace('+', ',', regex=False)

In [15]:
data_pi_zat = data_produk_pharos.copy()

#Cleaning data pi
data_pi_zat = data_pi_zat[data_pi_zat['Pack Molecule String'].isnull() == False]
data_pi_zat['Pack Molecule String'] = data_pi_zat['Pack Molecule String'].str.replace('\n', '', regex=False).str.replace(', ', ',', regex=False)
data_pi_zat['Pack Molecule String'] = data_pi_zat['Pack Molecule String'].str.replace('+', ',', regex=False)

In [18]:
#Memfilter data yang memiliki kode outlet dan kode dokter
data_dokter = data_dokter[(data_dokter['Kd Outlet'] != '-') & (data_dokter['Kd Dokter\n(DB Pharos)'] != '-')]
data_dokter = data_dokter.drop_duplicates()

#Menggabungkan data survey dengan data dokter berdasarkan kolom 'Doctor Name'
data_merged = pd.merge(data_survey_zat, data_dokter[['Kd Outlet','Doctor Name','Kd Dokter\n(DB Pharos)','Doctor Name \n(DB Pharos)','Kd Spesialis \n(DB Pharos)','Nama Spesialis\n(DB Pharos)']], on='Doctor Name', how='left')

data_merged = data_merged[data_merged['Kd Dokter\n(DB Pharos)'].isnull() == False]

#Membuat kolom Total
data_merged["Total"] = data_merged.iloc[:, 4:16].sum(axis=1)
data_merged = data_merged[data_merged['Total'] > 0]

# Menghapus baris yang mengandung kata 'retur' di nama produk
data_merged = data_merged[~data_merged['Product Name without #'].str.contains('retur', case=False, na=False)]

#Mengubah beberapa nama kolom
data_merged = data_merged.rename(columns = {
    'Kd Outlet' : "Kode RS",
    'Kd Dokter\n(DB Pharos)' : "Kode Dokter",
    'Doctor Name \n(DB Pharos)':"Nama Dokter",
    'Hospital Name':'Nama Rumah Sakit',
    'Kd Spesialis \n(DB Pharos)' : 'Kode Spesialis',
    'Nama Spesialis\n(DB Pharos)' : 'Nama Spesialis'
})

#Melakukan groupby berdasarkan kode rs, kode dokter, zat aktif
data_zat_aktif_dokter = data_merged.groupby(['Kode RS','Kode Dokter', 'Kode Spesialis','Pack Molecule String','Dosage Form'])['Total'].sum().reset_index()
data_dokter_atc = data_merged.groupby(['Kode RS','Kode Dokter', 'Kode Spesialis','ATC4','Dosage Form'])['Total'].sum().reset_index()

In [20]:
data_pi_merged = data_pi_zat.copy()
data_pi_merged = data_pi_merged.drop(['Best Match IMS Pack','Best Similarity Score','Kode Form','Jenis Form'], axis = 1)

## Recommendation Algorithm

### By Pack Molecule String

In [21]:
def rekomendasi_produk(data_dokter, data_produk):
    hasil_rekomendasi = []

    for _, row in data_zat_aktif_dokter.iterrows():
        kode_rs = row['Kode RS']
        kode_dokter = row['Kode Dokter']
        kode_spesialis = row['Kode Spesialis']
        zat_aktif = row['Pack Molecule String']
        total = row['Total']

        # Filter produk yang memiliki zat aktif yang sama
        produk_cocok = data_produk[data_produk['Pack Molecule String'] == zat_aktif]

        if not produk_cocok.empty:
            # Filter berdasarkan bentuk sediaan
            produk_sediaan_cocok = produk_cocok[produk_cocok['Dosage Form'] == row['Dosage Form']]

            if not produk_sediaan_cocok.empty:
                # Jika ada yang cocok berdasarkan sediaan, ambil yang pertama
                produk_terpilih = produk_sediaan_cocok.sort_values(by='Total Penjualan', ascending=False).iloc[0]
                metode_pemilihan = 'Zat Aktif & Bentuk Sediaan'
            else:
                # Jika tidak ada sediaan yang cocok, ambil produk dengan total sales tertinggi
                produk_terpilih = produk_cocok.sort_values(by='Total Penjualan', ascending=False).iloc[0]
                metode_pemilihan = 'Zat Aktif & Sales Tertinggi'

            hasil_rekomendasi.append({
                'Kode RS': kode_rs,
                'Kode Dokter': kode_dokter,
                'Kode Spesialis': kode_spesialis,
                'Zat Aktif': zat_aktif,
                'Produk Rekomendasi': produk_terpilih['Nama Produk'],
                'Kode Produk': produk_terpilih['Item Kode'],
                'Metode Pemilihan': metode_pemilihan,
                'Total Peresepan': total
            })

    return pd.DataFrame(hasil_rekomendasi)

produk_rekomendasi = rekomendasi_produk(data_zat_aktif_dokter, data_pi_merged)
produk_rekomendasi

,Kode RS,Kode Dokter,Kode Spesialis,Zat Aktif,Produk Rekomendasi,Kode Produk,Metode Pemilihan,Total Peresepan
0,A1005681,A1000684,003,"BACITRACIN,NEOMYCIN",NEBACETIN POWDER 5 G,000080,Zat Aktif & Bentuk Sediaan,1.0
1,A1005681,A1000684,003,CEFADROXIL,DROVAX 500 KAPSUL,007527,Zat Aktif & Bentuk Sediaan,11.0
2,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Zat Aktif & Bentuk Sediaan,1.0
3,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Zat Aktif & Sales Tertinggi,10.0
4,A1005681,A1000684,003,LOPERAMIDE,RHOMUZ 2 MG FC TABLET,013038,Zat Aktif & Bentuk Sediaan,2.0
...,...,...,...,...,...,...,...,...
33634,I1001005,Q1002060,011,MELOXICAM,MOXAM 7.5 MG TABLET,029637,Zat Aktif & Bentuk Sediaan,80.0
33635,I1001005,Q1002060,011,METHYLPREDNISOLONE,TORAS 8 MG TABLET,008792,Zat Aktif & Bentuk Sediaan,4.0
33636,I1001005,Q1002060,011,MOMETASONE,IFLACORT CREAM 10 G,008231,Zat Aktif & Sales Tertinggi,39.0
33637,I1001005,Q1002060,011,PARACETAMOL,PRAXION 250MG/5ML SUSPENSI,007549,Zat Aktif & Sales Tertinggi,3.0


In [22]:
data_copy_rekomendasi = produk_rekomendasi.copy()
# Manipulasi langsung di data_final untuk menggabungkan duplikat
for index, row in data_copy_rekomendasi.iterrows():
    duplikat_mask = (
        (data_copy_rekomendasi['Kode RS'] == row['Kode RS']) &
        (data_copy_rekomendasi['Kode Dokter'] == row['Kode Dokter']) &
        (data_copy_rekomendasi['Kode Produk'] == row['Kode Produk'])
    )
    matching_rows = data_copy_rekomendasi.loc[duplikat_mask]

    if len(matching_rows) > 1:
        bentuk_sediaan_mask = matching_rows['Metode Pemilihan'] == 'Zat Aktif & Bentuk Sediaan'
        if bentuk_sediaan_mask.any():
            total_sum = matching_rows['Total Peresepan'].sum()
            data_copy_rekomendasi.loc[duplikat_mask & bentuk_sediaan_mask, 'Total Peresepan'] = total_sum
            data_copy_rekomendasi = data_copy_rekomendasi.drop(matching_rows[~bentuk_sediaan_mask].index)
        else:
            total_sum = matching_rows['Total Peresepan'].sum()
            data_copy_rekomendasi.loc[duplikat_mask, 'Total Peresepan'] = total_sum
            data_copy_rekomendasi = data_copy_rekomendasi.drop(matching_rows.index[1:])

data_copy_rekomendasi

,Kode RS,Kode Dokter,Kode Spesialis,Zat Aktif,Produk Rekomendasi,Kode Produk,Metode Pemilihan,Total Peresepan
0,A1005681,A1000684,003,"BACITRACIN,NEOMYCIN",NEBACETIN POWDER 5 G,000080,Zat Aktif & Bentuk Sediaan,1.0
1,A1005681,A1000684,003,CEFADROXIL,DROVAX 500 KAPSUL,007527,Zat Aktif & Bentuk Sediaan,11.0
2,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Zat Aktif & Bentuk Sediaan,11.0
4,A1005681,A1000684,003,LOPERAMIDE,RHOMUZ 2 MG FC TABLET,013038,Zat Aktif & Bentuk Sediaan,2.0
5,A1005681,A1000684,003,METRONIDAZOLE,BIATRON 500 FC,007440,Zat Aktif & Sales Tertinggi,2.0
...,...,...,...,...,...,...,...,...
33634,I1001005,Q1002060,011,MELOXICAM,MOXAM 7.5 MG TABLET,029637,Zat Aktif & Bentuk Sediaan,80.0
33635,I1001005,Q1002060,011,METHYLPREDNISOLONE,TORAS 8 MG TABLET,008792,Zat Aktif & Bentuk Sediaan,4.0
33636,I1001005,Q1002060,011,MOMETASONE,IFLACORT CREAM 10 G,008231,Zat Aktif & Sales Tertinggi,39.0
33637,I1001005,Q1002060,011,PARACETAMOL,PRAXION 250MG/5ML SUSPENSI,007549,Zat Aktif & Sales Tertinggi,3.0


### By ATC4

In [23]:
def rekomendasi_produk(data_dokter, data_produk):
    hasil_rekomendasi = []

    for _, row in data_dokter_atc.iterrows():
        kode_rs = row['Kode RS']
        kode_dokter = row['Kode Dokter']
        kode_spesialis = row['Kode Spesialis']
        atc4 = row['ATC4']  # Menggunakan ATC4 sebagai acuan
        total = row['Total']

        # Filter produk yang memiliki ATC4 yang sama
        produk_cocok = data_produk[data_produk['ATC4'] == atc4]

        if not produk_cocok.empty:
            # Filter berdasarkan bentuk sediaan
            produk_sediaan_cocok = produk_cocok[produk_cocok['Dosage Form'] == row['Dosage Form']]

            if not produk_sediaan_cocok.empty:
                # Jika ada yang cocok berdasarkan sediaan
                produk_terpilih = produk_sediaan_cocok.sort_values(by='Total Penjualan', ascending=False).iloc[0]
                metode_pemilihan = 'ATC4 & Bentuk Sediaan'
            else:
                # Jika tidak ada sediaan yang cocok, ambil produk dengan total sales tertinggi
                produk_terpilih = produk_cocok.sort_values(by='Total Penjualan', ascending=False).iloc[0]
                metode_pemilihan = 'ATC4 & Sales Tertinggi'

            hasil_rekomendasi.append({
                'Kode RS': kode_rs,
                'Kode Dokter': kode_dokter,
                'Kode Spesialis': kode_spesialis,
                'ATC4': atc4,  # Menyimpan hasil pencocokan ATC4
                'Produk Rekomendasi': produk_terpilih['Nama Produk'],
                'Kode Produk': produk_terpilih['Item Kode'],
                'Metode Pemilihan': metode_pemilihan,
                'Total Peresepan': total
            })

    return pd.DataFrame(hasil_rekomendasi)

# Contoh pemanggilan fungsi
produk_rekomendasi_atc4 = rekomendasi_produk(data_dokter_atc, data_pi_merged)
produk_rekomendasi_atc4

,Kode RS,Kode Dokter,Kode Spesialis,ATC4,Produk Rekomendasi,Kode Produk,Metode Pemilihan,Total Peresepan
0,A1005681,A1000684,003,A02B1 - H2 ANTAGONISTS,POLYSILANE MAX TABLET (MINT),008935,ATC4 & Sales Tertinggi,1.0
1,A1005681,A1000684,003,A04A1 - SEROT ANTAG A-EMET+A-NAUS,NARFOZ 8 INJEKSI,005338,ATC4 & Bentuk Sediaan,2.0
2,A1005681,A1000684,003,A04A1 - SEROT ANTAG A-EMET+A-NAUS,NARFOZ 8 TABLET,003182,ATC4 & Bentuk Sediaan,1.0
3,A1005681,A1000684,003,A06A3 - BULK-FORMING LAXATIVES,MAXIMUS 500/45 MG CAPSULE,010850,ATC4 & Sales Tertinggi,1.0
4,A1005681,A1000684,003,A07H0 - MOTILITY INHIBITORS,RHOMUZ 2 MG FC TABLET,013038,ATC4 & Bentuk Sediaan,2.0
...,...,...,...,...,...,...,...,...
37449,I1001005,Q1002060,011,N02B0 - NON-NARCOTIC ANALGESICS,PRORIS SUSPENSI 60 ML,003259,ATC4 & Sales Tertinggi,3.0
37450,I1001005,Q1002060,011,N07C0 - ANTIVERTIGO PRODUCTS,DIZINE 10 MG TABLET,008704,ATC4 & Bentuk Sediaan,15.0
37451,I1001005,Q1002060,011,R05C0 - EXPECTORANTS,LIMOXIN 30 MG TAB,028669,ATC4 & Bentuk Sediaan,1034.0
37452,I1001005,Q1002060,011,R05D2 - ANTITUSSIVES COMB,OROXIN 1.65/33.3 MG/5 ML SYRUP 60 ML,012774,ATC4 & Sales Tertinggi,476.0


## Prepare Final Data

In [24]:
#Proses Cleaning product survey untuk membuat history produk

import re
def clean_product_name(product_name):
    if pd.isna(product_name):
        return ''
    # Hapus teks dalam tanda kurung
    product_name = re.sub(r'\(.*?\)', '', product_name)
    # Hapus teks setelah tanda pemisah seperti '-', ',' yang sering digunakan untuk zat aktif
    product_name = re.split(r' - |, ', product_name)[0]
    # Hapus spasi ekstra di awal dan akhir
    return product_name.strip()

data_history_dokter = data_merged[['Kode RS', 'Kode Dokter', 'Nama Dokter', 'Pack Molecule String', 'ATC4','Product Name without #']]
data_history_dokter['Product Name without #'] = data_history_dokter['Product Name without #'].apply(clean_product_name)

# Pengelompokkan history produk by zat aktif
data_history_dokter_zat_aktif = data_history_dokter.groupby(['Kode RS', 'Kode Dokter', 'Pack Molecule String'])['Product Name without #'].apply(
    lambda x: '; '.join(x.dropna().unique())
).reset_index()

# Pengelompokkan history produk by atc4
data_history_dokter_atc4 = data_history_dokter.groupby(['Kode RS', 'Kode Dokter', 'ATC4'])['Product Name without #'].apply(
    lambda x: '; '.join(x.dropna().unique())
).reset_index()


#Pembuatan kolom history produk by zat aktif
data_final = data_copy_rekomendasi.copy()
data_final = data_final.merge(
    data_history_dokter_zat_aktif,
    left_on = ['Kode RS','Kode Dokter','Zat Aktif'],
    right_on=['Kode RS','Kode Dokter', 'Pack Molecule String'],
    how='left'
).rename(columns={'Product Name without #': 'History Produk'})


#Pembuatan kolom history produk by ATC4
data_final_atc4 = produk_rekomendasi_atc4.copy()
data_final_atc4 = data_final_atc4.merge(
    data_history_dokter_atc4,
    left_on = ['Kode RS','Kode Dokter','ATC4'],
    right_on=['Kode RS','Kode Dokter', 'ATC4'],
    how = 'left'
).rename(columns={'Product Name without #': 'History Produk'})

#Membuat kolom status pssp

# Buat set untuk kombinasi unik dari kode dokter, kode produk, kode rs
pssp_set = set(zip(data_pssp['CD Cust'], data_pssp['CD Product'], data_pssp['CD Outlet']))

#Status PSSP by Zat Aktif
data_final['Status PSSP'] = data_final.apply(
    lambda row: 'Ya' if (row['Kode Dokter'], row['Kode Produk'], row['Kode RS']) in pssp_set else 'Tidak',
    axis=1)

#Status PSSP by ATC4
data_final_atc4['Status PSSP'] = data_final_atc4.apply(
    lambda row: 'Ya' if (row['Kode Dokter'], row['Kode Produk'], row['Kode RS']) in pssp_set else 'Tidak',
    axis=1)


#Membuat kolom status sales
# Buat set untuk kombinasi unik dari 'Item Kode' dan 'Kode PI' di data_standarisasi
standarisasi_set = set(zip(data_standarisasi['Item Kode'], data_standarisasi['KodePI']))

# Kolom status sales untuk hasil rekomendasi by zat aktif
data_final['Status Sales'] = data_final.apply(
    lambda row: 'Ya' if (row['Kode Produk'], row['Kode RS']) in standarisasi_set else 'Tidak',
    axis=1)

# Kolom status sales untuk hasil rekomendasi by atc4
data_final_atc4['Status Sales'] = data_final_atc4.apply(
    lambda row: 'Ya' if (row['Kode Produk'], row['Kode RS']) in standarisasi_set else 'Tidak',
    axis=1)


#Menampilkan nama dokter, nama rs, nama spesialis
data_dokter_ = data_merged[['Kode Dokter','Nama Dokter','Kode Spesialis','Nama Spesialis']].copy()
data_dokter_ = data_dokter_.drop_duplicates(subset=['Kode Dokter', 'Nama Dokter', 'Kode Spesialis', 'Nama Spesialis'])

#Implementasi ke dataset hasil rekomendasi berdasarkan zat aktif
data_final = data_final.merge(data_dokter_,
                              on = ['Kode Dokter','Kode Spesialis'],
                              how='left')

#Implementasi ke dataset hasil rekomendasi berdasarkan atc4
data_final_atc4 = data_final_atc4.merge(data_dokter_,
                              on = ['Kode Dokter','Kode Spesialis'],
                              how='left')


# MAPPING NAMA OUTLET
#Implementasi ke dataset rekomendasi by zat aktif
data_outlet = data_struktur_PI[['KodePI',"NamaOutlet"]].copy()
data_final = data_final.merge(data_outlet,
                              left_on = 'Kode RS',
                              right_on = 'KodePI',
                              how='left')
data_final = data_final.drop(['KodePI'], axis=1)

#Implementasi ke dataset rekomendasi by atc4
data_outlet = data_struktur_PI[['KodePI',"NamaOutlet"]].copy()
data_final_atc4 = data_final_atc4.merge(data_outlet,
                              left_on = 'Kode RS',
                              right_on = 'KodePI',
                              how='left')
data_final_atc4 = data_final_atc4.drop(['KodePI'], axis=1)


# Mengganti nama kolom (Rekomendasi by Zat Aktif)
data_final = data_final.rename(columns={
    'Total Peresepan': 'Potensi',
    'NamaOutlet': 'Nama RS',
    "Nama Spesialis" : 'Spesialis'
})
data_final = data_final[['Kode RS','Nama RS','Kode Dokter','Nama Dokter','Kode Produk','Produk Rekomendasi',
                         'Zat Aktif','Kode Spesialis','Spesialis','Potensi','History Produk',
                         'Status Sales','Status PSSP','Metode Pemilihan']]
# Mengganti nama kolom (Rekomendasi by ATC4)
data_final_atc4 = data_final_atc4.rename(columns={
    'Total Peresepan': 'Potensi',
    'NamaOutlet': 'Nama RS',
    "Nama Spesialis" : 'Spesialis'
})
data_final_atc4 = data_final_atc4[['Kode RS','Nama RS','Kode Dokter','Nama Dokter','Kode Produk','Produk Rekomendasi',
                         'ATC4','Kode Spesialis','Spesialis','Potensi','History Produk',
                         'Status Sales','Status PSSP','Metode Pemilihan']]

## Save to GDrive

### Result by Pack Molecule String

In [49]:
import datetime
import os

#Simpan langsung di GDrive
# Tentukan nama file berdasarkan tanggal saat ini
today_date = datetime.datetime.now().strftime('%d%m%Y')  # Format tanggal ddmmyyyy
file_name = f"{today_date} Data Rekomendasi Final.xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Rekomendasi"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
data_final.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Rekomendasi/13032025 Data Rekomendasi Final.xlsx


### Result by ATC4

In [50]:
#Simpan langsung di GDrive
# Tentukan nama file berdasarkan tanggal saat ini
today_date = datetime.datetime.now().strftime('%d%m%Y')  # Format tanggal ddmmyyyy
file_name = f"{today_date} Data Rekomendasi Final (ATC4).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Rekomendasi"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
data_final_atc4.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Rekomendasi/13032025 Data Rekomendasi Final (ATC4).xlsx
